This file represents the implementation of Facebook's bartMNLI used as an evaluation baseline for Multi-lbl2vec



In [ ]:
!pip install transformers
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model = "facebook/bart-large-mnli")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.3 MB/s 
     |████████████████████████████████| 182 kB 35.0 MB/s 
     |████████████████████████████████| 7.6 MB 12.8 MB/s 


Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
#Example of model output
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)['scores']

[0.9938651919364929, 0.0032737930305302143, 0.0028610294684767723]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import pandas as pd

In [ ]:
#Making sure manually labeled data is not null
from pandas.io.formats.format import NA
idxToStringMap = {0:"Ambience", 1:"Food", 2:"Service", 3:"Price", 4:"Time"}
stringToIdxMap ={"Ambience" : 0, "Food" : 1, "Service" : 2, "Price" : 3, "Time" : 4}

df = pd.read_csv("/content/drive/My Drive/CS 7650 Final Project/Data.csv")
testLabels = df.iloc[100 : 200][['Label-1', 'Label-2']]

labelStr = list(testLabels.itertuples(index=False, name = None))

checkedLabelsTest = []
for label in labelStr:
  firstTup = stringToIdxMap[label[0].strip()]

  if type(label[1]) == float:
    checkedLabelsTest.append((firstTup, ))
  
  else:
    secondTup = stringToIdxMap[label[1].strip()]
    checkedLabelsTest.append((firstTup, secondTup))

In [ ]:
import torch
import pandas as pd
def mnliModel(checkedLabelsTest):

  classScoresMatrix = torch.zeros(5, 3)

  df = pd.read_csv("/content/drive/My Drive/CS 7650 Final Project/Data.csv")

  reviews = df.iloc[100 : 200].text

  candidate_labels = ["Ambience", "Food", "Service", "Price", "Time"]

  currIdx = 0
  for review in reviews:
    model_output = torch.tensor(classifier(review, candidate_labels)['scores'])
    topIdx, secondIdx = torch.topk(model_output, 2)[1]
    topIdx = int(topIdx)
    secondIdx = int(secondIdx)
    topTwoTup = (topIdx, secondIdx)

    if len(checkedLabelsTest[currIdx]) > 1:

      for num in topTwoTup:
        if num in checkedLabelsTest[currIdx]:
          classScoresMatrix[num][0] += 1
        else:
          classScoresMatrix[num][1] += 0.5
      
      for num in checkedLabelsTest[currIdx]:
        if num not in topTwoTup:
          classScoresMatrix[num][2] += 0.5
    
    currIdx += 1

  print(classScoresMatrix)

  f1Scores = []
  for idx in range(len(classScoresMatrix)):
    precision = classScoresMatrix[idx][0] / (classScoresMatrix[idx][0] + classScoresMatrix[idx][1])
    recall = classScoresMatrix[idx][0] / (classScoresMatrix[idx][0] + classScoresMatrix[idx][2])

    f1Score = 0
    
    if (recall + precision) != 0:
      f1Score = (2 * recall * precision) / (recall + precision)

    f1Scores.append(f1Score)

    print("F1 Score for " + str(idxToStringMap[idx]) + ": " + str(float(f1Score)))


  print()
  print("Facebook MNLI Macro F1 Score : " + str(sum(f1Scores) / len(f1Scores) ))
  print()
  #print(data)

mnliModel(checkedLabelsTest)

Review:  0
tensor([0.9254, 0.0241, 0.0226, 0.0185, 0.0094])
Review:  1
tensor([0.4569, 0.2368, 0.1968, 0.0706, 0.0389])
Review:  2
tensor([0.7894, 0.1701, 0.0171, 0.0144, 0.0089])
Review:  3
tensor([0.8521, 0.0599, 0.0347, 0.0308, 0.0225])
Review:  4
tensor([0.5346, 0.1465, 0.1218, 0.1211, 0.0760])
Review:  5
tensor([0.5837, 0.3232, 0.0535, 0.0235, 0.0161])
Review:  6
tensor([0.5951, 0.2439, 0.0576, 0.0562, 0.0472])
Review:  7
tensor([0.5387, 0.1716, 0.1191, 0.1068, 0.0638])
Review:  8
tensor([0.4475, 0.2142, 0.1631, 0.0905, 0.0847])
Review:  9
tensor([0.8478, 0.0530, 0.0351, 0.0339, 0.0303])
Review:  10
tensor([0.4821, 0.2751, 0.0989, 0.0833, 0.0606])
Review:  11
tensor([0.8211, 0.0678, 0.0509, 0.0497, 0.0105])
Review:  12
tensor([0.6860, 0.1115, 0.0994, 0.0935, 0.0097])
Review:  13
tensor([0.3642, 0.3271, 0.2578, 0.0418, 0.0092])
Review:  14
tensor([0.5182, 0.3838, 0.0593, 0.0207, 0.0181])
Review:  15
tensor([0.6795, 0.1051, 0.0790, 0.0783, 0.0581])
Review:  16
tensor([0.4241, 0.3570